In [1]:
# -- coding: utf-8 -
import argparse
import os
import face_recognition
import numpy as np
import sklearn
import pickle
from face_recognition import face_locations
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
import cv2
import pandas as pd
import math

# import unicodedata
#cap = cv2.VideoCapture(0)
#leido, frame = cap.read()

frame = cv2.imread('Hombre.jpg')
frame = cv2.imread('Anteojos.png')
frame = cv2.imread('Asiatico.jpg')
frame = cv2.imread('Niño.jpeg')
frame = cv2.imread('Mujer.jpeg')
frame = cv2.imread('Bebe.jpeg')
frame = cv2.imread('Sindy1.jpeg')
frame = cv2.imread('Sindy2.jpeg')
frame = cv2.imread('Padre.jpeg')
frame = cv2.imread('Bebe1.jpeg')
frame = cv2.imread('p5.jpg')

#img = cv2.imread('Bebe1.jpg',cv2.IMREAD_GRAYSCALE )
#cv2.imshow('Imagen',img)

class PersonClassifier:

    def __init__(self):
        self.frame = frame
        self.race = None   
        self.gender = None
        self.age = None    
        self.ColorHair= None
        self.Glasses= None
        self.COLS = ['Male', 'Asian', 'White', 'Black',  'Baby', 'Child', 'Youth', 'Middle Aged', 'Senior', 'Black Hair', 'Blond Hair', 'Brown Hair', 'Bald', 'No eyewear', 'Eyeglasses', 'Sunglasses', 'Mustache', 'Smiling','Curly Hair', 'Wavy Hair', 'Straight Hair']
        self.N_UPSCLAE = 1
        self.model_path =('race_and_gender_model.pkl') # args.model

                #predecir una imagen
    def predict_one_image(self,img_path, clf, labels):
        print("2. predict_one_image")
        print(" ")

        """Predict face attributes for all detected faces in one image"""
        print("Predecir atributos de cara para todas las caras detectadas en una imagen.")
        
        face_encodings, locs = self.extract_features(img_path)
        #print(face_encodings, locs)
        
        
        if not face_encodings:
            return None, None
        pred = pd.DataFrame(clf.predict_proba(face_encodings),
                            columns = labels)
        #print(pred)
        pred = pred.loc[:, self.COLS]
        #print("Fin pred")
        print ("2.pred",pred)
        print ("2.locs",locs)
        print("_2. End predict_one_image ")
        print(" ")

        return pred, locs
        
    def gender_race(self,frame):
        with open(self.model_path) as f:

            clf, labels = pickle.load(f)
            print(clf)
            pred, locs = self.predict_one_image(frame, clf, labels)
            print(" ")
            print("pred")
            print(pred)
            locs = \
                pd.DataFrame(locs, columns = ['top', 'right', 'bottom', 'left'])
            df = pd.concat([pred, locs], axis=1)
            print("End pred")

            print(" ")
            #print("Imprimiendo DF")
            #print(df)
            img,gender_race = self.draw_attributes(frame, df)
        return gender_race
        print("0. gender_race")   
        #diccionario= {'Genero':None,'raza':None}
        
    #extraer_características
    def extract_features(self,img_path):
        print("3. extraer_características")
        print(" ")
        """Exctract 128 dimensional features"""
        print("Extraer 128 características dimensionales.")
        
        locs = face_locations(img_path, number_of_times_to_upsample = self.N_UPSCLAE)
        print(locs)
        if len(locs) == 0:
            return None, None
        face_encodings = face_recognition.face_encodings(frame, known_face_locations=locs)
        #print ("face_encodings",face_encodings)
        #print ("locs",locs)
        print("_3. End extract_features")

        return face_encodings, locs
 
    #dibujar atributos
    def draw_attributes(self,img_path, df):
        print(" ")
        print("4. dibujar atributos")
        print(" ")

        """Write bounding boxes and predicted face attributes on the image"""
        print("Escribe cuadros delimitadores y atributos de cara predichos en la imagen.")
        
        img = img_path 
        
        #nmedWindow( "Display window", CV_WINDOW_AUTOSIZE );
        #imshow( "Display window", image );
        #cv2.imread(img_path)
        #img2  = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
        for row in df.iterrows():
            caracteristicas=len(row[1])
            Car=caracteristicas-4
            print(row[1][Car:].astype(int))
            top, right, bottom, left = row[1][Car:].astype(int)
              
            print("row")
            print(type(row)) 
            print(row)
            print("")
            self.setGender(row[1]['Male'])
            self.setRace(np.argmax(row[1][1:4]))
            self.setAge(np.argmax(row[1][4:9]))
            self.setColorHair(np.argmax(row[1][9:13]))
            print("Asasasas")
            print(type(row[1][13]))
            self.setGlasses1(row[1][13],np.argmax(row[1][13:16]))
            text_showed = "{} {}".format(self.race, self.gender)
            #print (text_showed)
            Features = {'Gender':self.gender,'Race':self.race,'Age':self.age,'ColorHair':self.ColorHair,'Glasses':self.Glasses}
            cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)
            font = cv2.FONT_HERSHEY_DUPLEX
            img_width = img.shape[1]
            cv2.putText(img, text_showed, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)
            #print ("img",img)
            print ("4. Features",Features)
            print("_4. End draw_attributes ")
            print(" ")

        return img,Features
    
    def setGender(self,EstGender):

        if EstGender >= 0.5:
            self.gender = 'Male'
        else:
            self.gender = 'Female'

    def getGender(self):
        return self.gender

    def setRace(self,EstRace):
        self.race = EstRace

    def getRace(self):
        return self.race

    def setAge(self,EstAge):
        self.age = EstAge

    def getAge(self):
        return self.age

    def setColorHair(self,EstColorHair):
        self.ColorHair= EstColorHair

    def getColorHair(self):
        return self.ColorHair   
    
    def setGlasses1(self,NoEyewear,EstGlasses):
        if math.isnan(NoEyewear):
            self.Glasses= 'No Eyewear'
        else: 
            self.Glasses= EstGlasses
            
    def setGlasses(self,EstGlasses):
         self.Glasses= EstGlasses

    def getGlasses(self):
        return self.Glasses


    

PersonC=PersonClassifier()    
Features = PersonC.gender_race(frame)
print("Final Process")
print(" ")
#print('Gender:', PersonC.getGender() ,'race:',PersonC.getRace())
#print('Age:',PersonC.getAge(),'ColorHair:',PersonC.getColorHair())
#print('Glasses:',PersonC.getGlasses())


/home/jarvis-skynet/anaconda2/lib/python2.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.18.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jarvis-skynet/anaconda2/lib/python2.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.18.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=5000, momentum=0.9,
       n_iter_no_change=None, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.001,
       validation_fraction=0.1, verbose=True, warm_start=False)
2. predict_one_image
 
Predecir atributos de cara para todas las caras detectadas en una imagen.
3. extraer_características
 
Extraer 128 características dimensionales.
[(142L, 761L, 365L, 538L)]
_3. End extract_features
('2.pred',        Male     Asian     White     Black      Baby     Child     Youth  \
0  0.151399  0.010884  0.869486  0.006925  0.004379  0.060536  0.380958   

   Middle Aged    Senior  Black Hair  ...  Brown Hair    Bald  No eyewear  \
0     0.007703  0.036841    0.013076  ...    0.357605  0.0092         NaN   

   Eye

/home/jarvis-skynet/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/home/jarvis-skynet/anaconda2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


In [2]:
def EscogerMejor(df,keys):
    Dic=dict()
    for key in keys:
        if key !='Nombre':
            Col = [key,'Nombre']
            print(Col)
            param = df[Col].groupby([key]).count()
            param = param.sort_values(by='Nombre', ascending=False)
            param = param.index
            Dic[key]= param[0]
    return Dic

In [3]:
def ClasificadorDeFotos(tam):
    intro='p'
    ext='.jpg'
    cont=0
    dfList=list()
    tam=7 ##tam=len(Frame) 
    for index in range(tam):
        print(" ")
        print("ENTRENAMIENTO ", index+1)
        print(" ")
        nombre = intro + str(index+1) + ext
        
        PersonC=PersonClassifier()    
        img=cv2.imread(nombre,3)  ##Esto ya no iría
        Features = PersonC.gender_race(img) ##frame[i]
        Features['Nombre']=nombre
        print("Final Process")
        print(" ")
        df = pd.DataFrame([[Features[key] for key in Features.keys()]], columns= Features.keys())
        dfList.append(df)
        print(" ")
    df = pd.concat(dfList)
    Dic=EscogerMejor(df,Features.keys())
    PersonC.setGender(Dic['Gender'])
    PersonC.setAge(Dic['Age'])
    PersonC.setRace(Dic['Race'])
    PersonC.setColorHair(Dic['ColorHair'])
    PersonC.setGlasses(Dic['Glasses'])
    return PersonC

In [4]:
PersonC=ClasificadorDeFotos(7)

 
('ENTRENAMIENTO ', 1)
 
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=5000, momentum=0.9,
       n_iter_no_change=None, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.001,
       validation_fraction=0.1, verbose=True, warm_start=False)
2. predict_one_image
 
Predecir atributos de cara para todas las caras detectadas en una imagen.
3. extraer_características
 
Extraer 128 características dimensionales.
[(160L, 778L, 345L, 593L), (192L, 513L, 415L, 291L), (211L, 1013L, 340L, 884L)]
_3. End extract_features
('2.pred',        Male     Asian     White     Black      Baby     Child     Youth  \
0  0.111065  0.005960  0.902872  0.005938  0.003660  0.067637  0.353298   
1  0.820581  0.005261  0.846510  0.043870  0.005551  0.074604  0.184507   
2  0.971

[(617L, 2427L, 1576L, 1469L)]
_3. End extract_features
('2.pred',        Male     Asian     White     Black      Baby     Child     Youth  \
0  0.962937  0.013541  0.450804  0.022294  0.003697  0.037751  0.025679   

   Middle Aged    Senior  Black Hair  ...  Brown Hair      Bald  No eyewear  \
0     0.037243  0.040908    0.040377  ...    0.307573  0.079388         NaN   

   Eyeglasses  Sunglasses  Mustache  Smiling  Curly Hair  Wavy Hair  \
0      0.7917    0.030776  0.256754  0.23729    0.471787    0.43402   

   Straight Hair  
0        0.46811  

[1 rows x 21 columns])
('2.locs', [(617L, 2427L, 1576L, 1469L)])
_2. End predict_one_image 
 
 
pred
       Male     Asian     White     Black      Baby     Child     Youth  \
0  0.962937  0.013541  0.450804  0.022294  0.003697  0.037751  0.025679   

   Middle Aged    Senior  Black Hair  ...  Brown Hair      Bald  No eyewear  \
0     0.037243  0.040908    0.040377  ...    0.307573  0.079388         NaN   

   Eyeglasses  Sunglasses  Must

_3. End extract_features
('2.pred',        Male     Asian     White     Black      Baby     Child     Youth  \
0  0.164509  0.009328  0.879905  0.006509  0.005267  0.040981  0.221647   

   Middle Aged    Senior  Black Hair  ...  Brown Hair      Bald  No eyewear  \
0     0.018947  0.037667    0.020556  ...    0.431241  0.008556         NaN   

   Eyeglasses  Sunglasses  Mustache   Smiling  Curly Hair  Wavy Hair  \
0    0.044972    0.010769  0.010515  0.553084    0.537557   0.741617   

   Straight Hair  
0        0.14246  

[1 rows x 21 columns])
('2.locs', [(142L, 761L, 365L, 538L)])
_2. End predict_one_image 
 
 
pred
       Male     Asian     White     Black      Baby     Child     Youth  \
0  0.164509  0.009328  0.879905  0.006509  0.005267  0.040981  0.221647   

   Middle Aged    Senior  Black Hair  ...  Brown Hair      Bald  No eyewear  \
0     0.018947  0.037667    0.020556  ...    0.431241  0.008556         NaN   

   Eyeglasses  Sunglasses  Mustache   Smiling  Curly Hair  Wav

In [5]:
PersonC

<__main__.PersonClassifier instance at 0x7f051574af80>

In [6]:
Gender=    PersonC.getGender()
Age=    PersonC.getAge()
Race=    PersonC.getRace()
ColorHair=    PersonC.getColorHair()
Glasses=   PersonC.getGlasses()

In [7]:
Gender,Age,Race,ColorHair,Glasses

('Male', 'Senior', 'White', 'Brown Hair', 'No Eyewear')

In [191]:
df,keys=ClasificadorDeFotos(7)

 
('ENTRENAMIENTO ', 1)
 
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=5000, momentum=0.9,
       n_iter_no_change=None, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.001,
       validation_fraction=0.1, verbose=True, warm_start=False)
2. predict_one_image
 
Predecir atributos de cara para todas las caras detectadas en una imagen.
3. extraer_características
 
Extraer 128 características dimensionales.
[(160L, 778L, 345L, 593L), (192L, 513L, 415L, 291L), (211L, 1013L, 340L, 884L)]
_3. End extract_features
('2.pred',        Male     Asian     White     Black      Baby     Child     Youth  \
0  0.111065  0.005960  0.902872  0.005938  0.003660  0.067637  0.353298   
1  0.820581  0.005261  0.846510  0.043870  0.005551  0.074604  0.184507   
2  0.971

_3. End extract_features
('2.pred',        Male     Asian     White     Black      Baby     Child     Youth  \
0  0.962937  0.013541  0.450804  0.022294  0.003697  0.037751  0.025679   

   Middle Aged    Senior  Black Hair  ...  Brown Hair      Bald  No eyewear  \
0     0.037243  0.040908    0.040377  ...    0.307573  0.079388         NaN   

   Eyeglasses  Sunglasses  Mustache  Smiling  Curly Hair  Wavy Hair  \
0      0.7917    0.030776  0.256754  0.23729    0.471787    0.43402   

   Straight Hair  
0        0.46811  

[1 rows x 21 columns])
('2.locs', [(617L, 2427L, 1576L, 1469L)])
_2. End predict_one_image 
 
 
pred
       Male     Asian     White     Black      Baby     Child     Youth  \
0  0.962937  0.013541  0.450804  0.022294  0.003697  0.037751  0.025679   

   Middle Aged    Senior  Black Hair  ...  Brown Hair      Bald  No eyewear  \
0     0.037243  0.040908    0.040377  ...    0.307573  0.079388         NaN   

   Eyeglasses  Sunglasses  Mustache  Smiling  Curly Hair  Wav

('2.pred',        Male     Asian     White     Black      Baby     Child     Youth  \
0  0.164509  0.009328  0.879905  0.006509  0.005267  0.040981  0.221647   

   Middle Aged    Senior  Black Hair  ...  Brown Hair      Bald  No eyewear  \
0     0.018947  0.037667    0.020556  ...    0.431241  0.008556         NaN   

   Eyeglasses  Sunglasses  Mustache   Smiling  Curly Hair  Wavy Hair  \
0    0.044972    0.010769  0.010515  0.553084    0.537557   0.741617   

   Straight Hair  
0        0.14246  

[1 rows x 21 columns])
('2.locs', [(142L, 761L, 365L, 538L)])
_2. End predict_one_image 
 
 
pred
       Male     Asian     White     Black      Baby     Child     Youth  \
0  0.164509  0.009328  0.879905  0.006509  0.005267  0.040981  0.221647   

   Middle Aged    Senior  Black Hair  ...  Brown Hair      Bald  No eyewear  \
0     0.018947  0.037667    0.020556  ...    0.431241  0.008556         NaN   

   Eyeglasses  Sunglasses  Mustache   Smiling  Curly Hair  Wavy Hair  \
0    0.044972  

In [192]:
df

,Gender,Age,Race,Nombre,ColorHair,Glasses
0,Male,Child,White,p1.jpg,Brown Hair,No Eyewear
0,Male,Senior,White,p2.jpg,Brown Hair,No Eyewear
0,Male,Senior,White,p3.jpg,Brown Hair,No Eyewear
0,Male,Middle Aged,White,p4.jpg,Brown Hair,No Eyewear
0,Female,Youth,White,p5.jpg,Brown Hair,No Eyewear
0,Male,Senior,White,p6.jpg,Brown Hair,No Eyewear
0,Male,Senior,White,p7.jpg,Brown Hair,No Eyewear


In [193]:
keys

['Gender', 'Age', 'Race', 'Nombre', 'ColorHair', 'Glasses']

In [194]:
[Features[key] for key in Features.keys()]

['Female', 'Youth', 'White', 'Brown Hair', 'No Eyewear']

In [196]:
Dic=EscogerMejor(df,keys)
Dic

['Gender', 'Nombre']
['Age', 'Nombre']
['Race', 'Nombre']
['ColorHair', 'Nombre']
['Glasses', 'Nombre']


{'Age': 'Senior',
 'ColorHair': 'Brown Hair',
 'Gender': 'Male',
 'Glasses': 'No Eyewear',
 'Race': 'White'}

In [115]:
df.groupby(['Age']).count()

,Gender,Race,Nombre,ColorHair,Glasses
Age,,,,,
Child,1,1,1,1,1
Middle Aged,2,2,2,2,2
Senior,4,4,4,4,4


In [113]:
Age=df[['Age','Nombre']].groupby(['Age']).count()
Age=Age.sort_values(by='Nombre', ascending=False)
Age=Age.index
Age[0]

'Senior'

In [114]:
df[['Age','Nombre']].groupby(['Age']).max()

,Nombre
Age,
Child,p6.jpg
Middle Aged,p4.jpg
Senior,p7.jpg


In [22]:
df[['Age']].count()

7

In [32]:
df.groupby(['Age']).count()

,Gender,Race,Nombre,ColorHair,Glasses
Age,,,,,
Child,1,1,1,1,1
Middle Aged,2,2,2,2,2
Senior,4,4,4,4,4


In [ ]:
def ClasificadorDeFotos(tam):
    intro='p'
    ext='.jpg'
    cont=0
    dfList=list()
    tam=7
    for index in range(tam):
        print(" ")
        print("ENTRENAMIENTO ", index+1)
        print(" ")
        nombre = intro + str(index+1) + ext
        PersonC=PersonClassifier()    
        img=cv2.imread(nombre,3)
        Features = PersonC.gender_race(img)
        Features['Nombre']=nombre
        print("Final Process")
        print(" ")
        df = pd.DataFrame([[Features[key] for key in Features.keys()]], columns= Features.keys())
        dfList.append(df)
        print(" ")
    df = pd.concat(dfList)
    return Dic